In [2]:
import pandas as pd

df = pd.read_csv("label-full-metadata-20241219.csv")

df

,UUID,Domain,Parent Domain,Rating,Score,Last Updated,Country,Language,Does not repeatedly publish false or egregiously misleading content,Gathers and presents information responsibly,...,Twitter,Youtube,Instagram,LinkedIn,iOS App,Android App,Other,False Narratives,Communities Served,BrandGuard Segments
0,ea54ceae-86d9-4a08-9dd5-a383868e4513,womensagenda.com.au,NaN,T,80.0,1734562884151,AU,en,Yes,Yes,...,https://twitter.com/WomensAgenda,https://www.youtube.com/user/womensagenda/feat...,https://www.instagram.com/womensagenda/,https://www.linkedin.com/company/women%27s-agenda,NaN,NaN,https://www.pinterest.com.au/womensagenda/,NaN,NaN,"Inclusion - Basic Safety,Exclusion - Maximum S..."
1,8d01d1c4-2eb9-436a-9849-3a881249e3a5,kalminer.com.au,NaN,T,75.0,1734561522853,AU,en,Yes,Yes,...,https://twitter.com/kalgoorlieminer,NaN,https://www.instagram.com/kalgoorlieminer,https://www.linkedin.com/company/the-kalgoorli...,NaN,NaN,NaN,NaN,NaN,"Inclusion - Maximum Safety,Inclusion - High Sa..."
2,5d2a1da3-6ba7-44c9-bb9c-165864317a77,rightedition.com,NaN,N,20.0,1734561149837,CA,en,No,No,...,https://twitter.com/righteditionca,https://www.youtube.com/user/RightEditionNews,NaN,NaN,NaN,NaN,"https://odysee.com/@rightedition:5,https://www...","0ea16772-c048-41f9-b222-71b053f42ac8,9c4a3618-...",NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
3,d63f2e4f-7870-4bdf-9619-d08d823702c4,thefreethoughtproject.com,thefreethoughtproject.com,N,34.5,1734549146708,US,en,Yes,No,...,https://twitter.com/TFTPROJECT,https://www.youtube.com/channel/UCmT7hTV8VJU6C...,https://www.instagram.com/thefreethoughtproject/,NaN,NaN,NaN,NaN,"d7f3cb49-adef-4c0d-8f72-ba88018b53d5,3f675728-...",NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
4,d63f2e4f-7870-4bdf-9619-d08d823702c4,tftpnews.com,thefreethoughtproject.com,N,34.5,1734549146708,US,en,Yes,No,...,https://twitter.com/TFTPROJECT,https://www.youtube.com/channel/UCmT7hTV8VJU6C...,https://www.instagram.com/thefreethoughtproject/,NaN,NaN,NaN,NaN,"d7f3cb49-adef-4c0d-8f72-ba88018b53d5,3f675728-...",NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12462,5b442eb2-eb14-4cf0-b6ff-c81679685b0f,gesundheitstrends.com,NaN,T,82.5,1616835161457,DE,de,Yes,Yes,...,https://twitter.com/ges_trends,NaN,https://www.instagram.com/gesundheitstrendscom/,NaN,NaN,NaN,NaN,NaN,NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
12463,40fb1140-536a-47a0-a06c-023fbc3ddc07,flinkfeed.com,NaN,N,39.5,1615380820284,ALL,en,Yes,No,...,https://twitter.com/FlinkFeed_com,NaN,https://www.instagram.com/flinkfeed_com/,NaN,NaN,NaN,NaN,NaN,NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
12464,ae8b500d-683f-4e3d-a4a2-260b725ae142,flinkfeed.com,NaN,N,39.5,1615380816262,DE,de,Yes,No,...,https://twitter.com/FlinkFeed_com,NaN,https://www.instagram.com/flinkfeed_com/,NaN,NaN,NaN,NaN,NaN,NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."
12465,38ffb203-ebe5-4cac-8b5a-f84dd3697faf,blog.halle-leaks.de,NaN,N,37.5,1611208307207,DE,de,No,No,...,NaN,https://www.youtube.com/channel/UCspPfAQZ_txGk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Exclusion - Maximum Safety,Exclusion - High Sa..."


In [6]:
# Calculate average Score for each Rating type (T and N)
average_scores = df[df['Rating'].isin(['T', 'N'])].groupby('Rating')['Score'].mean()

average_scores

Rating
N    26.102152
T    87.720866
Name: Score, dtype: float64

In [7]:
# Get min score for T and max score for N
t_min = df[df['Rating'] == 'T']['Score'].min()
n_max = df[df['Rating'] == 'N']['Score'].max()

print(f"Minimum score for T: {t_min}")
print(f"Maximum score for N: {n_max}")

Minimum score for T: 60.0
Maximum score for N: 59.5


In [10]:
# Get all domains with Rating 'N' and their scores, sort by Score ascending
n_domains_sorted = df[df['Rating'] == 'N'][['Domain', 'Score']].sort_values('Score')

# Drop duplicates to keep only unique domains (keeping first occurrence which will be the lowest score)
n_domains_sorted = n_domains_sorted.drop_duplicates('Domain')

# Print the domains with their scores
print(f"Number of unique domains with Rating 'N': {len(n_domains_sorted)}")
print("\nDomains sorted by lowest Score:")
for _, row in n_domains_sorted.iterrows():
    print(f"{row['Domain']}: {row['Score']}")

Number of unique domains with Rating 'N': 4276

Domains sorted by lowest Score:
newsclub.pro: 0.0
money.whatfinger.com: 0.0
scitech.whatfinger.com: 0.0
sports.whatfinger.com: 0.0
nycsnews.pro: 0.0
breakingnewshouse.com: 0.0
thevoteinfo.com: 0.0
americanconservatives.today: 0.0
ajuanews.com: 0.0
nycpost.pro: 0.0
ntldrr.pro: 0.0
ntlddrr.com: 0.0
ntdr34.pro: 0.0
nrcnews.pro: 0.0
nndnews.pro: 0.0
newstv3.com: 0.0
2020maga.pro: 0.0
4thgenerationpolitics.com: 0.0
allaboutusa.pro: 0.0
allamericannews.pro: 0.0
altnewsonline.com: 0.0
americannews.pro: 0.0
newsservis.com: 0.0
newsrapter.com: 0.0
americasnewssource.com: 0.0
newspapercollections.com: 0.0
linkshare.whatfinger.com: 0.0
newsmedia2020.pro: 0.0
generaldispatch.whatfinger.com: 0.0
whatfinger.com: 0.0
todayupdates.pro: 0.0
mag24.cloud: 0.0
thecubenews.com: 0.0
texaspost.pro: 0.0
texasmag.pro: 0.0
seattledaily.site: 0.0
realdaily.pro: 0.0
rcidnews.pro: 0.0
politicsupdates.pro: 0.0
rightwingpost.com: 0.0
marialourdesblog.com: 0.0
phoenixda

In [66]:
import pandas as pd

def extract_untrusted_domains(csv_path, min_score=0, max_score=50):
    """
    Extract domains from NewsGuard CSV that are rated as untrusted ('N') 
    within a specified score range.
    
    Args:
        csv_path (str): Path to the NewsGuard CSV file
        min_score (int): Minimum score threshold (default: 10)
        max_score (int): Maximum score threshold (default: 30)
    
    Returns:
        list: List of domain names meeting the criteria
    """
    # Read the CSV
    df = pd.read_csv(csv_path)
    
    # Filter domains: Rating 'N' and scores within range
    filtered_domains = df[
        (df['Rating'] == 'N') & 
        (df['Score'] >= min_score) & 
        (df['Score'] <= max_score) &
        (df['Country'] == 'US') &
        (df['Language'] == 'en')
    ][['Domain', 'Score']].sort_values('Score')
    
    # Remove duplicates (keeping first occurrence which will have lowest score)
    filtered_domains = filtered_domains.drop_duplicates('Domain')
    
    # Convert to list
    return filtered_domains['Domain'].tolist()

# Example usage:
domains = extract_untrusted_domains("label-full-metadata-20241219.csv")
print(f"Found {len(domains)} domains")
print(domains)

Found 3378 domains
['thevoteinfo.com', 'newsrapter.com', 'newspapercollections.com', 'newsmedia2020.pro', 'newsmag.pro', 'newsleno.com', 'newsfeedin.com', 'newsclub.pro', 'news3insider.com', 'newnews40.pro', 'maxnews.pro', 'magnanewspapers.com', 'newsservis.com', 'maganews.pro', 'maga2020.pro', 'maga2020.press', 'ledinews.com', 'lapress.pro', 'jugadnews.com', 'iolgue.com', 'ilmaga.pro', 'homebuilders.space', 'hcworldnews.com', 'happymemorialday2019.com', 'hancevillenews.com', 'magafnb.press', 'newstv3.com', 'nndnews.pro', 'ntdr34.pro', 'washingtonpress.pro', 'usrightleft.com', 'uspodaily.pro', 'upmnetwork.com', 'txpolitico.site', 'todayupdates.pro', 'welovetrump.com', 'tmaga2020.pw', 'thecubenews.com', 'texaspost.pro', 'texasmag.pro', 'seattledaily.site', 'realdaily.pro', 'rcidnews.pro', 'politicsupdates.pro', 'phoenixdaily.club', 'peternews.xyz', 'pcsolottoresult4d.com', 'oregonnewspress.com', 'ohiopress.pro', 'nynews.pro', 'nycsnews.pro', 'nycpost.pro', 'ntldrr.pro', 'ntlddrr.com', '